In [1]:
%use krangl, lets-plot

In [2]:
class DecisionTree(private val maxDepth: Int = 5, private val featuresLimit: Boolean = false) {

    lateinit var x: List<DoubleArray>
    lateinit var y: List<Double>
    lateinit var w: List<Double>
    var root: Node? = null

    fun fit(x: List<DoubleArray>, y: List<Double>, weights: List<Double>) {
        this.x = x
        this.y = y
        this.w = weights
        root = trainNode(0, x.indices.toList())
    }

    fun predict(x: DoubleArray): Double {
        if (root == null) {
            throw RuntimeException("call fit first")
        }
        return root!!.predict(x)
    }

    private fun impurity(indices: List<Int>): Double {
        val sumLeaf = indices.sumByDouble { w[it] }
        val classesSumInLeaf = mutableMapOf<Double, Double>()
        for (ind in indices) {
            val cl = y[ind]
            classesSumInLeaf[cl] = classesSumInLeaf.getOrDefault(cl, 0.0) + w[ind]
        }
        return classesSumInLeaf.values.sumByDouble { 
            val norm = it / sumLeaf
            norm * (1 - norm)
        }
    }

    private fun qualityFunctional(left: List<Int>, right: List<Int>): Double {
        val fullSet = left + right
        val weight = { list: List<Int> -> list.sumByDouble { w[it] } }
        
        val fullSetWeight = weight(fullSet)
        val leftWeight = weight(left)
        val rightWeight = weight(right)
        
        return impurity(fullSet) - (leftWeight / fullSetWeight) * impurity(left) - (rightWeight / fullSetWeight) * impurity(right)
    }

    private fun trainNode(depth: Int, indices: List<Int>): Node {
        val curY = indices.map { y[it] }
        
        if (curY.distinct().size <= 1 || depth == maxDepth) {
            val classesSumInLeaf = mutableMapOf<Double, Double>()
            for (ind in indices) {
                val cl = y[ind]
                classesSumInLeaf[cl] = classesSumInLeaf.getOrDefault(cl, 0.0) + w[ind]
            }
            val pred = classesSumInLeaf.maxByOrNull { it.value }!!.key
            return LeafNode(pred)
        }

        var splitFeature = 0
        var splitThreshold = 0.0
        var maxQuality = -Double.MAX_VALUE
    
        var features = x.first().indices.toList()
        if (featuresLimit) {
            features = features.shuffled().take(sqrt(features.size.toDouble()).toInt())
        }
        for (featureInd in features) {
            val sortedIndices = indices.sortedBy { x[it][featureInd] }
        
            var prevTreshold = x[sortedIndices[0]][featureInd]
            for (ind in 1..sortedIndices.lastIndex) {
                val treshold = x[sortedIndices[ind]][featureInd]
                if (treshold == prevTreshold) {
                    continue
                } else {
                    prevTreshold = treshold
                }
                
                val left = sortedIndices.take(ind)
                val right = sortedIndices.drop(ind)
                val quality = qualityFunctional(left, right)

                if (quality > maxQuality) {
                    maxQuality = quality
                    splitFeature = featureInd
                    splitThreshold = treshold
                }
            }
        }

        val (left, right) = indices.partition { x[it][splitFeature] < splitThreshold }

        val leftNode = trainNode(depth + 1, left)
        val rightNode = trainNode(depth + 1, right)
        return SplitNode(splitFeature, splitThreshold, leftNode, rightNode)
    }

    interface Node {
        fun predict(x: DoubleArray): Double
    }

    class SplitNode(private val feature: Int, private val threshold: Double, private val left: Node, private val right: Node) : Node {
        override fun predict(x: DoubleArray): Double {
            return if (x[feature] < threshold) {
                left.predict(x)
            } else {
                right.predict(x)
            }
        }

    }

    class LeafNode(private val result: Double) : Node {
        override fun predict(x: DoubleArray): Double {
            return result
        }
    }
}

In [3]:
import kotlin.math.ln

class AdaBoost(val nSteps: Int = 8, val treeMaxDepth: Int = 1) {
    private val alphas: MutableList<Double> = mutableListOf()
    private val models: MutableList<DecisionTree> = mutableListOf()
        
    fun fit(x: List<DoubleArray>, y: List<Double>) {
        val w = MutableList(x.size) { 1.0 / x.size }
    
        for (step in 0 until nSteps) {
            val dt = DecisionTree(treeMaxDepth)
            dt.fit(x, y, w)
            
            val iErrors = List(y.size) { i -> if (y[i] != dt.predict(x[i])) 1.0 else 0.0 }
            
            var error = 0.0
            for (i in w.indices) {
                error += w[i] * iErrors[i]
            }
            error /= w.sum()
            
            val alpha = ln((1.0 - error) / error)
            
            var z = 0.0
            for (i in w.indices) {
                z += w[i] * exp(alpha * iErrors[i])
            }
            
            for (i in w.indices) {
                w[i] = w[i] * exp(alpha * iErrors[i])
            }
            alphas.add(alpha)
            models.add(dt)
        }
    }
    
    fun predict(x: DoubleArray): Double {
        var y = 0.0
        for (i in models.indices) {
            y += alphas[i] * models[i].predict(x)
        }
        return if (y > 0) 1.0 else -1.0
    }
}

In [4]:
fun DataFrame.toShuffledXY(): Pair<List<DoubleArray>, List<Double>> {
    val objs = this.rows.toList().map { it.values.map { it.toString() } }.shuffled()
    return Pair(objs.map { it.dropLast(1).map { it.toDouble() }.toDoubleArray() }, objs.map { if (it.last() == "P") 1.0 else -1.0 })
}

In [5]:
fun accuracy(x: List<DoubleArray>, y: List<Double>, boostSteps: Int = 8, maxTreeSize: Int = 1): Double {
    val batchSize = x.size / 4
    val accuracies = mutableListOf<Double>()
    
    for (startInd in x.indices step batchSize) {
        val finInd = min(startInd + batchSize, x.size)
        
        val testX = x.subList(startInd, finInd)
        val trainX = x.filterIndexed { ind, _ -> ind !in startInd until finInd }
        val testY = y.subList(startInd, finInd)
        val trainY = y.filterIndexed { ind, _ -> ind !in startInd until finInd }
        
        val adaBoost = AdaBoost(boostSteps, maxTreeSize)
        adaBoost.fit(trainX, trainY)
        
        var correctPredictions = 0
        for (testObjInd in testX.indices) {
            val pred = adaBoost.predict(testX[testObjInd])
            if (pred == testY[testObjInd]) {
                correctPredictions++
            }
        }
        accuracies.add(correctPredictions.toDouble() / testX.size)
    }
    
    return accuracies.mean()
}

In [6]:
val boostingStepsCount = listOf(1, 2, 3, 5, 8, 13, 21, 34, 55)
val treeMaxSize = 2

In [7]:
val chipsDataset = DataFrame.readCSV("data/chips.csv")
val (chipsX, chipsY) = chipsDataset.toShuffledXY()
val chipsAcc = boostingStepsCount.map { accuracy(chipsX, chipsY, it, treeMaxSize) }

val chipsP = lets_plot() + geom_line { x=boostingStepsCount; y=chipsAcc }
chipsP.show()

In [21]:
val cellCount = 50
var xs = List(cellCount * cellCount) { i -> -1.1 + 2.2 * ((i % cellCount).toDouble() / (cellCount - 1)) }
var ys = List(cellCount * cellCount) { i -> -1.1 + 2.2 * ((i / cellCount).toDouble() / (cellCount - 1)) }

for (stepsCount in boostingStepsCount) {
    println(stepsCount)
    val ada = AdaBoost(stepsCount, treeMaxSize)
    ada.fit(chipsX, chipsY)

    var z = xs.zip(ys).map { 
        ada.predict(doubleArrayOf(it.first, it.second))
    }

    var plotInfo = mapOf(
        "x" to chipsX.map { it.first() },
        "y" to chipsX.map { it[1] },
        "color" to chipsY
    )

    var p1 = lets_plot {x=xs; y=ys; fill=z}
    p1 += geom_tile() + 
        scale_fill_hue() + 
        geom_point {  x=plotInfo["x"]; y=plotInfo["y"]; fill=x; color=plotInfo["color"] }
    p1.show()
}

1


2


3


5


8


13


21


34


55


In [23]:
val geyserDataset = DataFrame.readCSV("data/geyser.csv")
val (geyserX, geyserY) = geyserDataset.toShuffledXY()
val geyserAcc = boostingStepsCount.map { accuracy(geyserX, geyserY, it, treeMaxSize) }

val geyserP = lets_plot() + geom_line { x=boostingStepsCount; y=geyserAcc }
geyserP.show()

In [29]:
val cellCount = 50
xs = List(cellCount * cellCount) { i -> 25 * ((i % cellCount).toDouble() / (cellCount - 1)) }
ys = List(cellCount * cellCount) { i -> 1.0 + 5.0 * ((i / cellCount).toDouble() / (cellCount - 1)) }

for (stepsCount in boostingStepsCount) {
    println(stepsCount)
    val ada = AdaBoost(stepsCount, treeMaxSize)
    ada.fit(geyserX, geyserY)

    var z = xs.zip(ys).map { 
        ada.predict(doubleArrayOf(it.first, it.second))
    }

    var plotInfo = mapOf(
        "x" to geyserX.map { it.first() },
        "y" to geyserX.map { it[1] },
        "color" to geyserY
    )

    var p1 = lets_plot {x=xs; y=ys; fill=z}
    p1 += geom_tile() + 
        scale_fill_hue() + 
        geom_point {  x=plotInfo["x"]; y=plotInfo["y"]; fill=x; color=plotInfo["color"] }
    p1.show()
}

1
2


3
5


8


13
21


34


55
